In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [9]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/bert-sentiment-analysis/data/processed/train_processed.csv')
val_df = pd.read_csv('/content/drive/MyDrive/bert-sentiment-analysis/data/processed/val_processed.csv')
test_df = pd.read_csv('/content/drive/MyDrive/bert-sentiment-analysis/data/processed/test_processed.csv')

In [10]:
from transformers import BertTokenizer
# Initialize the pretrained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
def tokenize_texts(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length', # pad shorter texts to max_length
        truncation=True, # truncate longer texts
        max_length=max_length,
        return_tensors='pt' # return PyTorch tensors
    )
# Tokenize cleaned sentences for each split
train_encodings = tokenize_texts(train_df['cleaned_sentence'])
val_encodings = tokenize_texts(val_df['cleaned_sentence'])
test_encodings = tokenize_texts(test_df['cleaned_sentence'])

In [12]:
import torch
# Convert labels to torch tensors
train_labels = torch.tensor(train_df['label'].values)
val_labels = torch.tensor(val_df['label'].values)
test_labels = torch.tensor(test_df['label'].values)

In [13]:
from torch.utils.data import Dataset
# Create Dataset class
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

# Create datasets for train, val, and test
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [14]:
from transformers import BertForSequenceClassification

# Load base pretrained BERT model and classification head (3 labels - negative, neutral, positive)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Set up optimizer and learning rate scheduler

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

batch_size = 16
num_epochs = 3

optimizer = AdamW(model.parameters(), lr=5e-5)

num_training_steps = len(train_dataset) // batch_size * num_epochs

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [9]:
# Upgrade transformer to latest version
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 107.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [16]:
# Confirm the version of transformer
import transformers
print(transformers.__version__)
print(transformers.__file__)

4.53.2
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [3]:
import os
print(os.listdir('.'))

['.config', 'drive', 'sample_data']


In [4]:
!pip uninstall -y transformers

Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2


In [1]:
!pip install transformers==4.53.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [6]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    eval_strategy='epoch'
)

print(args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

In [4]:
import transformers
print(transformers.__version__)
print(transformers.__file__)

from transformers import TrainingArguments
print(TrainingArguments.__module__)
print(TrainingArguments.__file__ if hasattr(TrainingArguments, '__file__') else 'No __file__ attribute')

4.53.2
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py
transformers.training_args
No __file__ attribute


In [5]:
import inspect
from transformers import TrainingArguments

print(inspect.signature(TrainingArguments.__init__))

(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str, NoneType] = <factory>, warmup_ratio: float = 0.0, 

In [19]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/bert-financial-sentiment-classifier/results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16, # training batch size per device
    per_device_eval_batch_size=16,  # evaluation batch size
    eval_strategy='epoch',     # evaluate at the end of each epoch
    save_strategy='epoch',           # save checkpoint every epoch
    learning_rate=5e-5,              # learning rate
    weight_decay=0.01,               # weight decay
    logging_dir='/content/drive/MyDrive/bert-financial-sentiment-classifier/logs',            # logging directory
    logging_steps=50,                # log every 50 steps
    load_best_model_at_end=True,    # load best model at end of training
    metric_for_best_model='accuracy' # metric to use for best model selection
)

# Define metric computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-19-3704577386.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.521500,0.541435,0.755705,0.768427,0.813518,0.755705
2,0.284100,0.595131,0.766443,0.775856,0.792335,0.766443


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.521500,0.541435,0.755705,0.768427,0.813518,0.755705
2,0.284100,0.595131,0.766443,0.775856,0.792335,0.766443
3,0.213000,0.645472,0.758389,0.762035,0.767050,0.758389


TrainOutput(global_step=792, training_loss=0.3971877760357327, metrics={'train_runtime': 19541.3894, 'train_samples_per_second': 0.648, 'train_steps_per_second': 0.041, 'total_flos': 832753967109120.0, 'train_loss': 0.3971877760357327, 'epoch': 3.0})

In [20]:
model.save_pretrained('/content/drive/MyDrive/bert-financial-sentiment-classifier/model')
tokenizer.save_pretrained('/content/drive/MyDrive/bert-financial-sentiment-classifier/model')

('/content/drive/MyDrive/bert-financial-sentiment-classifier/model/tokenizer_config.json',
 '/content/drive/MyDrive/bert-financial-sentiment-classifier/model/special_tokens_map.json',
 '/content/drive/MyDrive/bert-financial-sentiment-classifier/model/vocab.txt',
 '/content/drive/MyDrive/bert-financial-sentiment-classifier/model/added_tokens.json')

In [22]:
import os

model_dir = '/content/drive/MyDrive/bert-financial-sentiment-classifier/model'
print("Exists:", os.path.exists(model_dir))
print("Contents:", os.listdir(model_dir) if os.path.exists(model_dir) else "Folder not found")

Exists: True
Contents: ['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt']


In [24]:
import torch
import pandas as pd

# Load processed test data CSV
test_df = pd.read_csv('/content/drive/MyDrive/bert-financial-sentiment-classifier/data/processed/test_processed.csv')


# Tokenize test texts (make sure tokenizer is initialized)
test_encodings = tokenizer(
    list(test_df['cleaned_sentence']),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Convert labels to tensor
test_labels = torch.tensor(test_df['label'].values)

# Create Dataset object
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)


test_dataset = SentimentDataset(test_encodings, test_labels)

# Evaluate model
metrics = trainer.evaluate(test_dataset)
print(metrics)


KeyError: 'cleaned_sentence'

In [27]:
import pandas as pd

test_df = pd.read_csv('/content/drive/MyDrive/bert-financial-sentiment-classifier/data/processed/test_processed.csv')

print("Columns:", test_df.columns)
print(test_df.head())


Columns: Index(['Sentence', 'Sentiment', 'label'], dtype='object')
                                            Sentence Sentiment  label
0  The number of bodily injury cases quadrupled i...  negative      0
1  Net sales decreased to EUR 91.6 mn from EUR 10...   neutral      1
2   $aapl high of day just hit. Back at it tomorrow.  positive      2
3  According to CEO Kai Telanne , the company 's ...  positive      2
4  Finland 's dominating rail company VR is plann...   neutral      1


In [31]:
print(test_df.head())

                                            Sentence Sentiment  label  \
0  The number of bodily injury cases quadrupled i...  negative      0   
1  Net sales decreased to EUR 91.6 mn from EUR 10...   neutral      1   
2   $aapl high of day just hit. Back at it tomorrow.  positive      2   
3  According to CEO Kai Telanne , the company 's ...  positive      2   
4  Finland 's dominating rail company VR is plann...   neutral      1   

                                    cleaned_sentence  
0    The number of bodily injury cases quadrupled in  
1  Net sales decreased to EUR  mn from EUR mn in ...  
2           high of day just hit Back at it tomorrow  
3  According to CEO Kai Telanne  the company s ne...  
4  Finland s dominating rail company VR is planni...  


In [30]:
import re

def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

# Add cleaned_sentence if missing
if 'cleaned_sentence' not in test_df.columns:
    test_df['cleaned_sentence'] = test_df['Sentence'].apply(clean_text)

In [32]:
test_encodings = tokenizer(
    list(test_df['cleaned_sentence']),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

In [33]:
import torch

test_labels = torch.tensor(test_df['label'].values)

In [34]:
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

In [35]:
test_dataset = SentimentDataset(test_encodings, test_labels)

In [36]:
metrics = trainer.evaluate(test_dataset)
print(metrics)

{'eval_loss': 0.5291472673416138, 'eval_accuracy': 0.8027366020524516, 'eval_f1': 0.8093768490883736, 'eval_precision': 0.823593416797985, 'eval_recall': 0.8027366020524516, 'eval_runtime': 386.8626, 'eval_samples_per_second': 2.267, 'eval_steps_per_second': 0.142, 'epoch': 3.0}
